# Text segmentation using Hidden Markov Models

```yaml
Name: Jiale KANG, Shujian YANG
Date: Mar 15, 2024
```

# Q1:

**Question:**

Give the value of the π vector of the initial probabilities:

<div class='alert alert-block alert-warning'>
            Answer:</div>

Knowing that each email begins with a header, this means the probability of starting in state 1 is $1$, and the probability of starting in state 2 is $0$.

Therefore, the initial probability vector $\pi$ is:
\begin{equation*}
    \pi = 
    \begin{bmatrix}
        1\\
        0
    \end{bmatrix}
\end{equation*}


# Q2

**Question:**

What is the probability to move from state 1 to state 2 ? What is the probability
to remain in state 2 ? What is the lower/higher probability ? Try to explain why: 

<div class='alert alert-block alert-warning'>
            Answer:</div>

The probability of moving from state 1 to state 2 is $A(1,2)=0.000781921964187974$.

The probability of staying in state 2 is $A(2,2)=1$.

The probability of remaining in state 2 is the higher probability compared to the probability of moving from state 1 to state 2. This is logical considering the structure of an email: once you finish the header and start the body, you do not go back to the header. Hence, there is a 100% chance of staying in the body (state 2) once it is reached, which is reflected by the transition probability of 1. On the other hand, the probability of moving from the header to the body is very high but not absolute, allowing for multiple lines in the header before the transition to the body occurs. The small value of $0.000781921964187974$ indicates that while the transition from the header to the body does occur, it is very rare or happens after a large number of observations within the header, reflecting the possibility of a long header section.

# Q3

**Question:**

 What is the size of B ?


<div class='alert alert-block alert-warning'>
            Answer:</div>

$B \in \mathbb{R}^{2\times N}$

In [1]:
import os
import glob
import numpy as np

In [2]:
ROOT = os.path.abspath('.')

PERL_DIR = os.path.join(ROOT,'PerlScriptAndModel')
RES_DIR = os.path.join(ROOT,'res')

### Coding/Decoding Mails

In [3]:
DATA_DIR = os.path.join(ROOT,'dat')

# Iterate through files and load the text 
def files_iter(data_dir, with_name=False):
    files = glob.glob('{}/*.dat'.format((data_dir)))
    if with_name:
        for f in files:
            # Get the filename 
            name = f.split('/')[-1].split('.')[0]
            # Return filename and associated text
            yield name, np.loadtxt(f, dtype=int)
    else:
        for f in files :
            yield np.loadtxt(f, dtype=int)

In [4]:
# And we get a generator that will allow us to iterate through the mails
mail_iter = files_iter(DATA_DIR, with_name=True)

### Distribution files

In [5]:
PERL_DIR = os.path.join(ROOT,'PerlScriptAndModel')

# Writing a function to get the probability data
def get_emission_prob(perl_dir):
    return np.loadtxt(os.path.join(perl_dir, 'P.text'))

In [6]:
# Inputs to the Viterbi function
trans = np.array([[0.999218078035812, 0.000781921964187974], [0, 1]])
emission_prob = get_emission_prob(PERL_DIR)
states = list(range(trans.shape[0]))
start_prob = np.array([1, 0])

### To implement:

In [7]:
# Viterbi function
def viterbi(obs, states, start_prob, trans, emission_prob):
    """
        Viterbi Algorithm Implementation

        Keyword arguments:
            - obs: sequence of observation
            - states:list of states
            - start_prob:vector of the initial probabilities
            - trans: transition matrix
            - emission_prob: emission probability matrix
        Returns:
            - seq: sequence of state
    """

    # Avoid underflow: use the logarithm !
    # Avoid 0 in logarithm: use a small constant !
    small = np.finfo(float).tiny
    
    start_prob = np.log(start_prob + small)
    trans = np.log(trans + small)
    emission_prob = np.log(emission_prob + small)
    
    T = len(obs)
    N = len(states)
    
    # Initialisation
    log_l = np.empty((T, N), "d")
    bcktr = np.zeros((T, N), 'int')
    
    # Viterbi
    
    # Forward loop:
    log_l[0,:]= start_prob + emission_prob[obs[0],:]
    for t in range(1, T):
        log_l[t, :] = np.max(log_l[t - 1, :] + trans.T + emission_prob[obs[t], :].T, 1)
        bcktr[t, :] = np.argmax(log_l[t - 1, :] + trans.T + emission_prob[obs[t], :], 1)
    # Backward loop
    path = np.zeros(T, 'int')
    path[-1] = np.argmax(log_l[-1, :])

    for t in reversed(range(1,T)):
        path[t-1] = bcktr[t, path[t]]
    return path+1

In [8]:
RES_DIR = os.path.join(ROOT,'res')

# Creating a directory to put the result of the viterbi function
if not os.path.exists(RES_DIR):
    os.mkdir(RES_DIR)
    
# Function that will write a viterbi path for a mail in a dedicated result file
def create_viterbi_path_file(mail_name, viterbi_path):
    with open('{}/{}_path.txt'.format(RES_DIR, mail_name), 'w') as f: 
        f.write(''.join([str(c) for c in viterbi_path]))   

In [9]:
# Using our generator, we get the mail names and data
for name_file, data in mail_iter:
    # Find out the viterbi path using viterbi
    viterbi_path = viterbi(data, states, start_prob, trans, emission_prob)
    # print(name_file, viterbi_path)
    # Put it in the result file
    create_viterbi_path_file(name_file, viterbi_path)

### Visualizing segmentation

In [10]:
# Writing a function to go into the directory and execute the perl script "segment.pl" on the mail in the given path
def exec_perl_script(mail, path):
    res = !cd {PERL_DIR}; perl segment.pl {mail} {path}
    return res

# Writing a function getting the original mail, the result of viterbi, and applying the segmentation script
# Then putting the result
def segment_mail(mail_name, data_dir, output_dir):
    # Get the full path of the mail
    mail = os.path.join(data_dir, '{}.txt'.format(mail_name))
    # Get the full path of the result
    path = os.path.join(output_dir, '{}_path.txt'.format(mail_name))

    # Execute the visualization script
    formatted_mail = exec_perl_script(mail, path)
    # Get the results
    formatted_mail_text = formatted_mail[1:]

    # Go through the resulting text until the cutting line
    cutting_line = '========================== coupez ici =========================='
    for i in range(len(formatted_mail_text)):
        if formatted_mail_text[i] == cutting_line:
            segment_idx = i
            break
    # If this was not the last line, return the text cut in to parts: header and body
    if segment_idx != len(formatted_mail_text):
        header = formatted_mail_text[:segment_idx]
        body = formatted_mail_text[segment_idx+1:]
    # If not, it's just a header
    else:
        header = formatted_mail_text
        body = None

    print("++++++++++++++++++++[Header]++++++++++++++++++++\n")
    for line in header:
        print(line + '\n')
    if body:
        print("++++++++++++++++++++[Body]++++++++++++++++++++\n")
        for line in body:
            print(line + '\n')


In [11]:
# Getting mails names
# segment_mail("mail11",DATA_DIR,RES_DIR)
mail_iter = files_iter(DATA_DIR, with_name=True)
for name_file, data in mail_iter:
    print(name_file)
    segment_mail(name_file, DATA_DIR, RES_DIR)
# Call the function and look at the result of segmentation
# ...  

mail21
++++++++++++++++++++[Header]++++++++++++++++++++

From timc@2ubh.com  Thu Aug 22 17:31:00 2002

Return-Path: <timc@2ubh.com>

Delivered-To: zzzz@localhost.netnoteinc.com

Received: from localhost (localhost [127.0.0.1])

	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id A97BE43F99

	for <zzzz@localhost>; Thu, 22 Aug 2002 12:30:58 -0400 (EDT)

Received: from phobos [127.0.0.1]

	by localhost with IMAP (fetchmail-5.9.0)

	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 17:30:58 +0100 (IST)

Received: from n17.grp.scd.yahoo.com (n17.grp.scd.yahoo.com

    [66.218.66.72]) by dogma.slashnull.org (8.11.6/8.11.6) with SMTP id

    g7MGOFZ14344 for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 17:24:16 +0100

X-Egroups-Return: sentto-2242572-52744-1030033454-zzzz=spamassassin.taint.org@returns.groups.yahoo.com

Received: from [66.218.67.201] by n17.grp.scd.yahoo.com with NNFMP;

    22 Aug 2002 16:24:16 -0000

X-Sender: timc@2ubh.com

X-Apparently-To: zzzzteana@yahoogroups.com